In [ ]:
!pip install pytimekr

In [ ]:
import pandas as pd
import numpy as np
import time
import datetime as dt
from pytimekr import pytimekr
import os
import glob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
import ast

In [ ]:
path='C:\\Users\smhrd\머신러닝_김세나' # csv파일 위치경로
file_list = glob.glob(os.path.join(path, 'insta_train_*')) # insta_train_으로 시작하는 파일 모음
print(file_list) # 확인하기

In [ ]:
## csv 파일들을 DataFrame으로 불러와서 concat
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트
for file in file_list:
    df = pd.read_csv(file, engine='python') # for구문으로 csv파일 읽어들임
    allData.append(df) # 빈 리스트에 읽어 들인 내용 추가

In [ ]:
dataCombine = pd.concat(allData, axis=0, ignore_index=True) # concat함수를 이용해서 리스트의 내용병합

In [ ]:
dataCombine.to_csv("insta_합침.csv", index=False, encoding = 'euc-kr') # 합친 csv저장 

In [ ]:
insta = pd.read_csv('insta_합침.csv',encoding="euc-kr",index_col=False)  
insta  

In [ ]:
insta = insta.drop_duplicates() # 중복제거
insta

In [ ]:
upload_day = insta.iloc[:,3] # 업로드 날짜 
upload_day

In [ ]:
# 시간대 만들기
upload_time_list = []
upload_time = insta.iloc[:,4]
for time in upload_time:
    r_time = time.replace(":",'')
    upload_time_list.append(int(r_time[:2]))
insta["업로드시간대"] = upload_time_list

In [ ]:
# 업로드 시간대를 0시~1시 는 1시, 2시~3시는 3시 이런식으로 바꿔주는 함수 bin함수하고 비슷

def bining_time(x):
    if 2>x>= 0:
        return 1
    elif 4>x >= 2:
        return 3
    elif 6> x >=4:
        return 5
    elif 8> x >= 6:
        return 7
    elif 10>x >= 8:
        return 9
    elif 12>x >= 10:
        return 11
    elif 14>x >= 12 :
        return 13
    elif 16>x >= 14:
        return 15
    elif 18>x>=16:
        return 17
    elif 20>x >= 18:
        return 19
    elif 22>x >= 20:
        return 21
    else :
        return 23

In [ ]:
insta['업로드시간대'] = insta['업로드시간대'].apply(bining_time)

In [ ]:
# 업로들 날짜를 요일로 변경
total_day_week = ["월요일",'화요일','수요일','목요일','금요일','토요일','일요일']
day_week_list = []
for day in upload_day:
    day_ = dt.datetime.strptime(day,'%Y-%m-%d')
    weekday_ = dt.datetime.weekday(day_)
    
    day_week_list.append(total_day_week[weekday_])
insta['업로드요일'] = day_week_list

In [ ]:
# 공휴일 유무 (어린이날,석가탄신일) 차후 다른 공휴일 있으면 넣어주면됨(?)
# 5302 오류가 나오는것같다 이부분은 확인하기
insta['공휴일유무'] = None
for i in range(len(upload_day)):
    if upload_day[i] in '2021-05-05': # 어린이날
        insta['공휴일유무'][i]= 1
    elif upload_day[i] in '2021-05-19': # 석가탄신일
        insta['공휴일유무'][i] = 1
    elif upload_day[i] in '2021-06-06': # 현충일
        insta['공휴일유무'][i] = 1
    elif upload_day[i] in '2021-03-01': # 삼일절
        insta['공휴일유무'][i] = 1
    else : 
        insta['공휴일유무'][i] = 0 

In [ ]:
# 필요없는 컬럼 삭제(업로드 날짜, 업로드 시간)
del insta["업로드 날짜"]
del insta["업로드 시간"]

In [ ]:
insta

In [ ]:
hash_num = insta.iloc[:,5] # 해시태그 
hash_num

In [ ]:
# 해시개수 추가
hash_num_list = []
for i in hash_num :
    num_ = i.replace("[","").replace("]","").replace("'","").split(',')
    hash_num_list.append(num_)
insta['해시태그개수'] =hash_num_list

In [ ]:
for i in range (len(hash_num_list)) :
    insta['해시태그개수'][i] = len(hash_num_list[i])

In [ ]:
insta

In [ ]:
insta['해시태그'] = insta['해시태그'].apply(lambda x: ast.literal_eval(x))
# df를 불러왔을 때 list를 값으로 갖는 column은 string 형태로 불러와지므로 list로 바꿔줌

In [ ]:
insta['해시태그']

In [ ]:
insta

In [ ]:
corpus = []
for i in insta['해시태그']:
    corpus.append(' '.join(i))

In [ ]:
corpus

In [ ]:
c = {}
for i in insta['해시태그']:
    for word in i:
        if word in c:
            c[word] += 1
        else:
            c[word] = 1

In [ ]:
c # 해시태그의 등장 횟수 딕셔너리

In [ ]:
c.values()

In [ ]:
cnt = 0
valueList = []
for i,j in zip(c.keys(),c.values()):
    if j<=10 :  
        valueList.append(i)

In [ ]:
cnt = 0
cntList = []
for hashtag in insta['해시태그'] :
    for tag in hashtag:
        if tag in valueList:
            cntList.append(cnt)
            break
    cnt+=1   
cntList       

In [ ]:
insta = insta.drop(index=cntList, axis=0)

In [ ]:
insta

In [ ]:
insta.to_csv("insta_광고제거까지.csv", index=False, encoding = 'euc-kr')

In [ ]:
insta = pd.read_csv("insta_광고제거까지.csv",  encoding = 'euc-kr')

In [ ]:
# !pip install cmake
# !pip install dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

ALL = list(range(0, 68)) 
RIGHT_EYEBROW = list(range(17, 22))  
LEFT_EYEBROW = list(range(22, 27))  
RIGHT_EYE = list(range(36, 42))  
LEFT_EYE = list(range(42, 48))  
NOSE = list(range(27, 36))  
MOUTH_OUTLINE = list(range(48, 61))  
MOUTH_INNER = list(range(61, 68)) 
JAWLINE = list(range(0, 17)) 
index = ALL

In [ ]:
user_face_size = []
user_face_number = []

In [ ]:
for i in tqdm(insta.loc[:,'사진 이름']) :
    
    cnt = 0
    
    face_size_list = []
    
    img_frame = cv.imread("./insta_img/{}".format(i))
    img_gray = cv.cvtColor(img_frame, cv.COLOR_BGR2GRAY)

    dets = detector(img_gray, 1)

    for face in dets:
        shape = predictor(img_frame, face) #얼굴에서 68개 점 찾기

        list_points = []
        for p in shape.parts():
            list_points.append([p.x, p.y])
        list_points = np.array(list_points)

        for i,pt in enumerate(list_points[index]):
            pt_pos = (pt[0], pt[1])
            cv.circle(img_frame, pt_pos, 2, (0, 255, 0), -1)

        cv.rectangle(img_frame, (face.left(), face.top()), (face.right(), face.bottom()),
            (0, 0, 255), 3)
        face_size_list.append( (face.right()-face.left())*(face.bottom()-face.top()) )
        cnt+=1
    try:
        max_size = max(face_size_list)
    except :
        max_size = 0
    
    user_face_number.append(cnt)
    user_face_size.append(max_size)

In [ ]:
insta.to_csv("insta_dlib.csv", index=False, encoding = 'euc-kr')